In [1]:
import random
import heapq
from dataclasses import dataclass, field
from typing import List, Tuple, Callable, Any, Set
import numpy as np
from time import time
from tabulate import tabulate
import threading
import collections
import itertools
import copy
from collections import deque
from typing import List, Tuple
from itertools import combinations
from math import ceil
from enum import Enum
from functools import partial
from copy import deepcopy


<h1 style="color:#D40078; font-weight:bold;">Initial State:</h1>

<span style="color: blue"> The user provides a matrix representing the puzzle's initial state, where each cell corresponds to a lamp in the room. The coordinates (-1, -1) serve as the tree's root, representing an abstract starting point before any actions have been taken.</span>.


<h1 style="color:#D40078; font-weight:bold;">Actions:</h1>
<span style="color: blue">For each node in the tree, the child nodes are determined by selecting puzzle cells with coordinates greater than the current node's coordinates. This approach ensures that the tree is finite by restricting the potential actions to a limited set of valid moves.</span>.

<h1 style="color:#D40078; font-weight:bold;">Goal:</h1>
<span style="color: blue">The puzzle's goal is to turn off all the lamps in the room.The search through the tree continues until this condition is met.</span>.

<h1 style="color:orange; ">Node Class:</h1>
<span style="color: blue">Attributes:</span>.




<span style="color: orange">Position (x, y):</span> <span style="color: blue">The coordinates of the node in the puzzle matrix, representing the position of a lamp.</span>.

<span style="color: orange">Cost (g):</span>. <span style="color: blue">The cost from the root node to this node, representing the number of nodes seen so far.</span>.

<span style="color: orange">Heuristic (h): </span><span style="color: blue">An estimate of the cost to reach the goal from the current node. This value is problem-specific and depends on how the heuristic function is defined (e.g., the number of lamps still on).</span>

<span style="color: orange">Evaluation Function (f):</span>. <span style="color: blue">The sum of the cost and the heuristic (f = g + h). This value is used to prioritize nodes in informed search algorithms like A*</span>.

<span style="color: orange">Methods:</span>
<span style="color: blue">The Node class also includes methods to handle the generation of child nodes and other functionalities:</span>.

<span style="color: orange">get_child():</span>. <span style="color: blue">This method generates child nodes based on the current node's position (x, y). A child node corresponds to a puzzle cell whose coordinates are greater than the current node's coordinates. The method ensures that the tree is finite by only generating valid child nodes that represent a forward movement in the puzzle.</span>


In [2]:

class LightsOutPuzzle:
    def __init__(self, board: List[List[int]]):
        self.board = np.array(board, dtype=np.uint8)
        self.size = len(board)

    def toggle(self, x, y):
        for dx, dy in [(0, 0), (1, 0), (-1, 0), (0, 1), (0, -1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                self.board[nx, ny] ^= 1

    def is_solved(self):
        return np.all(self.board == 0)
        
    def copy(self):
        return LightsOutPuzzle(self.board.tolist())

    def get_moves(self):
        return [(x, y) for x in range(self.size) for y in range(self.size)]

    def __str__(self):
        return '\n'.join(' '.join(str(cell) for cell in row) for row in self.board)


    def print_board(self):
        for row in self.board:
            print(" ".join(map(str, row)))
        print("\n" + "-" * 10 + "\n")


In [3]:
def create_random_board(size: int,  seed: int, num_toggles: int = None):
    random.seed(time() if seed is None else seed)

    board = [[0 for _ in range(size)] for _ in range(size)]
    puzzle = LightsOutPuzzle(board)

    if num_toggles is None:
        num_toggles = random.randint(1, size * size)

    for _ in range(num_toggles):
        x = random.randint(0, size - 1)
        y = random.randint(0, size - 1)
        puzzle.toggle(x, y)

    return puzzle.board

In [4]:
def show_solution(
    puzzle: LightsOutPuzzle,
    solution: List[Tuple[int, int]],
    algorithm_name: str,
    nodes_visited: int,
    show_steps: bool = False,
):
    print(f"\nSolving with {algorithm_name}:")
    if solution is None:
        print("No solution found.")
        return
    print(f"Solution: {solution}")
    print(f"Nodes visited: {nodes_visited}")
    if show_steps:
        print("\nSolution steps:")
        for i, move in enumerate(solution):
            print(f"\nStep {i+1}: Toggle position {move}")
            puzzle.toggle(*move)
            print(puzzle)
        print(
            "\nFinal state (solved):"
            if puzzle.is_solved()
            else "\nFinal state (not solved):"
        )
        print(puzzle)

In [5]:
tests = [
    create_random_board(3, 42),
    create_random_board(3, 41),
    create_random_board(3, 42, 5),
    create_random_board(4, 42),
    create_random_board(4, 41),
    create_random_board(4, 42, 5),
    create_random_board(5, 42),
    create_random_board(5, 41),
    create_random_board(5, 42, 5),
]

In [6]:
class Nodes:
    def __init__(self, row=0, col=0, n=3,  h=0, g=0):
        self.x = row
        self.y = col
        self.n = n
        self.h = h
        self.g = g
        self.f = g + h 

    def get_children(self): 
        
        coordinates = [(i, j) for i in range(self.n) for j in range(self.n)]
        if self.x == -1 and self.y == -1:
            return coordinates

        current_position = (self.x, self.y)
        return [coord for coord in coordinates if coord > current_position]


<h1 style="color:#D40078; font-weight:bold;">BFS:</h1>
<span style="color:blue;">The Breadth-First Search (BFS) algorithm explores all the nodes at the present depth level before moving on to nodes at the next depth level. I begin by examining all the nodes at the first depth level. </span>
<span style="color:blue;">Once I have traversed these nodes, I proceed to the nodes at the second depth level and continue this process until we reach our goal.</span>

<span style="color:blue;">To implement this algorithm, I utilize a queue. When I reach a node, I append its child nodes to the end of the queue along <span style="color:blue;">with the updated state of the board after toggling at that node and  moving sequence until this node. This ensures that nodes are processed in a FIFO manner.</span>

<span style="color:orange; font-weight:bold">Visited Nodes: </span><span style="color:blue;">In my implementation, I do not track visited nodes since my tree is finite. I can guarantee that there are no repeated combinations of nodes.</span>

In [7]:
def bfs_solve(puzzle: LightsOutPuzzle) -> Tuple[List[Tuple[int, int]], int]:
    queue = deque([(puzzle.copy(), [], [(-1, -1)])])
    visited_boards = set()
    visited_boards.add(tuple(map(tuple, puzzle.board)))
    visited_nodes = 1
    while queue:
        current_board, moves, node = queue.popleft()
        visited_nodes += 1
        
        for x, y in node:
                current_board.toggle(x, y)

        if current_board.is_solved():
            return moves, visited_nodes

        new_node = Nodes(node[0][0], node[0][1], n=puzzle.size)
        
        for neighbor in new_node.get_children():
            if current_board not in visited_boards:
                    visited_boards.add(tuple(map(tuple, current_board.board)))
                    queue.append((current_board.copy(), moves + [neighbor], [neighbor]))

   
    return None, visited_nodes
    

<h1 style="color:#D40078; font-weight:bold;">IDS:</h1> <span style="color:blue;">To implement IDS, I first implemented a limited depth search algorithm using a recursive function. This function takes a depth limit as an argument and, until the depth limit reaches zero, it calls itself recursively. At the beginning of each call, the function checks if the puzzle is solved. If the depth limit is zero, it stops the recursion and does not explore further. After that, I wrote the IDS algorithm, which starts with a depth of 0 and increments the depth in a loop. The loop continues until a solution is found. Once the solution is found, the loop exits.</span>



In [8]:
def depth_limited_search(puzzle: LightsOutPuzzle, depth_limit: int):
    nodes_count = 1  
    result, count = recursive_dls(puzzle, -1, -1, depth_limit)
    nodes_count += count
    return result, nodes_count

def recursive_dls(puzzle: LightsOutPuzzle, initial_x: int, initial_y: int, depth_limit: int):
    if puzzle.is_solved():
        return [], 0

    if depth_limit == 0:
        return "cutoff", 0
    node = Nodes(initial_x, initial_y, n=puzzle.size)
    moves = []
    nodes_count = 0
    cutoff_occurred = False

    for child in node.get_children():
        puzzle.toggle(child[0], child[1])
        moves.append(child)
        nodes_count += 1
        result, count = recursive_dls(puzzle, child[0], child[1], depth_limit - 1)
        nodes_count += count

        if result != "cutoff":
            if result is not None:
                return moves + result, nodes_count

        puzzle.toggle(child[0], child[1])
        moves.pop()

        if result == "cutoff":
            cutoff_occurred = True

    if cutoff_occurred:
        return "cutoff", nodes_count
    else:
        return None, nodes_count


def ids_solve(puzzle: LightsOutPuzzle) -> Tuple[List[Tuple[int, int]], int]:
    depth_limit = 0
    total_nodes = 0

    while True:
        solution_moves, nodes_count = depth_limited_search(puzzle, depth_limit)
        total_nodes += nodes_count

        if solution_moves is not None and solution_moves != "cutoff":
            return solution_moves, total_nodes
        depth_limit += 1


test_puzzle = LightsOutPuzzle(tests[0])
solution_moves, total_nodes = ids_solve(test_puzzle)
print("Moves to solve:", solution_moves)
print("Total nodes visited:", total_nodes)

Moves to solve: [(0, 2), (1, 0)]
Total nodes visited: 31


<h1 style="color:#D40078; font-weight:bold;">A*:</h1> <span style="color:blue;">To implement A*, we need a heuristic. The first heuristic I used is the number of lights that are still on. In this implementation, I first add the children of each node to an open list. Then, I sort the list based on the `f` value (where `f = g + h`, with `g` being the cost to reach the node (number of the move that I visited) and `h` being the heuristic). I expand the first node in the queue, calculate the new heuristic for its children, and check if the new puzzle state is solved or not.</span>

<span style="color:blue;">Another heuristic is the Manhattan distance between each node and all other nodes with a value of 1.</span> 

In [9]:
def astar_solve(puzzle: LightsOutPuzzle, heuristic: Callable[[LightsOutPuzzle], int]) -> Tuple[List[Tuple[int, int]], int]:
    open_list = []
    closed_list = []
    initial_node = Nodes(-1, -1, n=puzzle.size, g =0, h=heuristic(puzzle))
    initial_board = puzzle.copy()
    open_list.append((initial_node, initial_board, []))
    visited_nodes  = 1
    while open_list:
        open_list.sort(key = lambda x: x[0].f)
        current_node, current_board, moves = open_list.pop(0)
        #print(f"current node: {current_node.x, current_node.y}")
        #print(f"moves: {moves}")
        #current_node.g += 1 
        #current_board.print_board()
        

        #if current_board.is_solved():
        #    return moves, visited_nodes
        for child in current_node.get_children():
            new_board = current_board.copy()
            new_board.toggle(child[0], child[1])
            #print(f"inner loop child: {child}")
            #print(f"inner loop child heuristic: {heuristic(new_board)}")
            new_node = Nodes(child[0], child[1], n=puzzle.size, g = current_node.g + 1 , h=heuristic(new_board))
            #print(f"inner loop child f: {new_node.f}")
            visited_nodes += 1
            if new_board.is_solved():
                return moves + [child], visited_nodes 
            open_list.append((new_node, new_board, moves +[child]))
                
    return None, visited_nodes


#print(tests[0])
#tes1 = [[1, 0], [0, 1]] 
#print(tes1)
#test_puzzle = LightsOutPuzzle(tests[0])

#solution_moves, total_nodes = astar_solve(test_puzzle, heuristic1)
#print("Moves to solve:", solution_moves)
#print("Total nodes visited:", total_nodes)

In [10]:
def heuristic1(puzzle: LightsOutPuzzle):
    count = 0; 
    for row  in puzzle.board:
        for light in row:
            if light == 1:
                count += 1
      
    return count


def heuristic2(puzzle: LightsOutPuzzle) -> int:
    board = puzzle.board
    size = len(board)
    on_lights = [(i, j) for i in range(size) for j in range(size) if board[i][j] == 1]
    total_distance = 0
    for i in range(len(on_lights)):
        for j in range(i + 1, len(on_lights)):
            x1, y1 = on_lights[i]
            x2, y2 = on_lights[j]
            total_distance += abs(x1 - x2) + abs(y1 - y2)
    return total_distance

heuristics = [heuristic1, heuristic2]


<h1 style="color:#D40078; font-weight:bold;">A* weighted:</h1> <span style="color:blue;">To implement weighted A*, we simply multiply the heuristic by a weight. I used weights of 2 and 4 for the heuristic (denoted as alpha).</span>
<span style="color:blue;">Another heuristic is the Manhattan distance between each node and all other nodes with a value of 1.</span>


In [11]:
def weighted_heuristic1(puzzle: LightsOutPuzzle, alpha = 5):
  return heuristic1(puzzle) * alpha
def weighted_heuristic2(puzzle: LightsOutPuzzle, alpha = 5):
  return heuristic2(puzzle) * alpha


weighted_heuristics = [weighted_heuristic1, weighted_heuristic2]
weights = [2, 4]

In [12]:
class SearchStatus(Enum):
    SOLVED = "Solved"
    TIMEOUT = "Timeout"
    NO_SOLUTION = "No Solution"

@dataclass
class SearchResult:
    result: SearchStatus
    solution: Any = None
    nodes_visited: int = None
    time: float = None

    def __str__(self):
        output = f"Result: {self.result.value}\n"
        if self.result == SearchStatus.SOLVED:
            output += f"Solution: {self.solution}\n"
        if self.result != SearchStatus.TIMEOUT:
            output += f"Nodes Visited: {self.nodes_visited}\n"
            output += f"Time Taken: {self.time:.3f} seconds\n"

        return output


DEFAULT_SEARCH_RESULT = SearchResult(SearchStatus.TIMEOUT, None, None, float("inf"))

<h1 style="color:#D40078; font-weight:bold;">Comparing Different Methods:</h1> <span style="color:blue;">Compared to BFS, IDS is a better search algorithm because it uses less memory while still being complete and optimal.</span> <span style="color:blue;">A*, which is an informed search algorithm, outperforms uninformed algorithms if our heuristic is admissible and effective. As we can see, the first heuristic is better than the second one.</span>


In [13]:
def run_searches(func: Callable, args: Any, time_limit: float = 60.0, name: str = "") -> SearchResult:
    result = []

    def target():
        try:
            solution, nodes_visited = func(*args)
            result.append((solution, nodes_visited))
        except Exception as e:
            result.append(e)

    thread = threading.Thread(target=target)
    thread.start()

    start_time = time()
    thread.join(timeout=time_limit)

    if thread.is_alive() or not result:
        print(f"\nTime limit of {time_limit} seconds exceeded for {name}")
        return SearchResult(SearchStatus.TIMEOUT)

    if isinstance(result[0], Exception):
        raise result[0]

    solution, nodes_visited = result[0]
    show_solution(args[0], solution, name, nodes_visited)
    time_taken = time() - start_time
    return SearchResult(SearchStatus.SOLVED if solution else SearchStatus.NO_SOLUTION, solution, nodes_visited, time_taken)

In [14]:
def run_test(test: List, heuristics: List[Callable[[Any], int]], time_limit: float = 60.0):

    print(f"Running test:\n {test}")

    puzzle = LightsOutPuzzle(test)

    bfs_result = run_searches(bfs_solve, (deepcopy(puzzle),), name="BFS", time_limit=time_limit)
    ids_result = run_searches(ids_solve, (deepcopy(puzzle),), name="IDS", time_limit=time_limit)

    astar_results = []
    for heuristic in heuristics:
        astar_results.append(
            run_searches(
                astar_solve,
                (deepcopy(puzzle), heuristic),
                name=f"A*({heuristic.__name__})",
                time_limit=time_limit,
            )
        )

    weighted_astar_results = []
    for heuristic in weighted_heuristics:
        for weight in weights:
          weighted_astar_results.append(
              run_searches(
                  astar_solve,
                  (deepcopy(puzzle), partial(heuristic, alpha = weight)),
                  name=f"weighted A*({heuristic.__name__}, alpha = {weight})",
                  time_limit=time_limit,
              )
          )

    print("\n-------------------------------\n")

    return (
        test,
        bfs_result,
        ids_result,
        *astar_results,
        *weighted_astar_results
    )

In [15]:
results: List[Tuple[str, SearchResult, SearchResult, SearchResult, SearchResult]] = []

for test in tests:
    results.append(run_test(test, heuristics, 180)\')

Running test:
 [[1 1 1]
 [1 1 1]
 [1 0 0]]

Solving with BFS:
Solution: [(0, 2), (1, 0)]
Nodes visited: 27

Solving with IDS:
Solution: [(0, 2), (1, 0)]
Nodes visited: 31

Solving with A*(heuristic1):
Solution: [(0, 2), (1, 0)]
Nodes visited: 36

Solving with A*(heuristic2):
Solution: [(0, 2), (1, 0)]
Nodes visited: 34

Solving with weighted A*(weighted_heuristic1, alpha = 2):
Solution: [(0, 2), (1, 0)]
Nodes visited: 51

Solving with weighted A*(weighted_heuristic1, alpha = 4):
Solution: [(0, 2), (1, 0)]
Nodes visited: 64

Solving with weighted A*(weighted_heuristic2, alpha = 2):
Solution: [(0, 2), (1, 0)]
Nodes visited: 34

Solving with weighted A*(weighted_heuristic2, alpha = 4):
Solution: [(0, 2), (1, 0)]
Nodes visited: 37

-------------------------------

Running test:
 [[1 1 0]
 [1 0 0]
 [1 1 1]]

Solving with BFS:
Solution: [(0, 0), (0, 1), (1, 0), (1, 1), (1, 2)]
Nodes visited: 273

Solving with IDS:
Solution: [(0, 0), (0, 1), (1, 0), (1, 1), (1, 2)]
Nodes visited: 471

Solving

In [16]:
!pip install tabulate

In [18]:
table_data: List[List[str]] = []

for result in results:
    table_data.append(
        [
            "\n".join(" ".join(str(cell) for cell in row) for row in result[0]),
            str(result[1]),
            str(result[2]),
        ]
    )

    for i in range(len(heuristics)):
        table_data[-1].append(str(result[3 + i]))

    for i in range(len(weighted_heuristics)):
        for j in range(len(weights)):
          table_data[-1].append(str(result[3 + len(heuristics) + i * 2 + j]))

    table_data.append("\n\n\n")

if table_data[-1] == "\n\n\n":
    table_data.pop()

print(len(table_data))
print("Results:")
print(
    tabulate(
        table_data,
        headers=[
            "Test",
            "BFS",
            "IDS",
            *[f"A* ({heuristic.__name__})" for heuristic in heuristics],
            *[f"Weighted A* ({heuristic.__name__}, alpha = {weight})" for heuristic in weighted_heuristics for weight in weights]
        ],
        floatfmt=".3f",
        numalign="center",
        stralign="center",
        tablefmt="grid",
    )
)

17
Results:
+-----------+----------------------------------------------------+----------------------------------------------------+--------------------------------------------------------------------+------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+
|   Test    |                        BFS                         |                        IDS                         |                          A* (heuristic1)                           |                      A* (heuristic2)                       |                            Weighted A* (weighted_heuristic1, alpha = 2)                            |                            Weighted A* (weighted_heuristic1, al